***GENERATED CODE FOR cachemanualfe PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'City'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '327', 'mean': '', 'stddev': '', 'min': 'ARVADA', 'max': 'LITTLETON', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'transformation_label': 'String Indexer', 'feature_label': 'City'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Product_Category'}], 'feature': 'Product_Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Category'}, {'transformation_label': 'String Indexer', 'feature_label': 'Product_Category'})
        transformationDF = transformationDF.drop('Product_Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Product_Sub_Category'}], 'feature': 'Product_Sub_Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'Appliances', 'max': 'Telephones and Communication', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Sub_Category'}, {'transformation_label': 'String Indexer', 'feature_label': 'Product_Sub_Category'})
        transformationDF = transformationDF.drop('Product_Sub_Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Customer_Segment'}], 'feature': 'Customer_Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Small Business', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer_Segment'}, {'transformation_label': 'String Indexer', 'feature_label': 'Customer_Segment'})
        transformationDF = transformationDF.drop('Customer_Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'First_Name'}], 'feature': 'First_Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'ADXXXX', 'max': 'YVXXXX', 'missing': '0', 'distinct': '96'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'First_Name'}, {'transformation_label': 'String Indexer', 'feature_label': 'First_Name'})
        transformationDF = transformationDF.drop('First_Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Last_Name'}], 'feature': 'Last_Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'ACXXXX', 'max': 'ZIXXXXXXX', 'missing': '0', 'distinct': '116'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Last_Name'}, {'transformation_label': 'String Indexer', 'feature_label': 'Last_Name'})
        transformationDF = transformationDF.drop('Last_Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Address'}], 'feature': 'Address', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': '1011 S VALENTIA ST', 'max': '974 RACINE ST', 'missing': '0', 'distinct': '125'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Address'}, {'transformation_label': 'String Indexer', 'feature_label': 'Address'})
        transformationDF = transformationDF.drop('Address')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'State'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'CO', 'max': 'CO', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'transformation_label': 'String Indexer', 'feature_label': 'State'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'MOSAIC_HOUSEHOLD'}], 'feature': 'MOSAIC_HOUSEHOLD', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'A01', 'max': 'U00', 'missing': '0', 'distinct': '42'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC_HOUSEHOLD'}, {'transformation_label': 'String Indexer', 'feature_label': 'MOSAIC_HOUSEHOLD'})
        transformationDF = transformationDF.drop('MOSAIC_HOUSEHOLD')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'MOSAIC_DESCRIPTION'}], 'feature': 'MOSAIC_DESCRIPTION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'Autumn Years', 'max': 'Young City Solos', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC_DESCRIPTION'}, {'transformation_label': 'String Indexer', 'feature_label': 'MOSAIC_DESCRIPTION'})
        transformationDF = transformationDF.drop('MOSAIC_DESCRIPTION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Channel'}], 'feature': 'Channel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '327', 'mean': '', 'stddev': '', 'min': 'Catalog', 'max': 'eCommerce', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Channel'}, {'transformation_label': 'String Indexer', 'feature_label': 'Channel'})
        transformationDF = transformationDF.drop('Channel')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run cachemanualfeHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': '216.168.41.46', 'port': '19872', 'password': 'num_sql@1433', 'database': 'numtra', 'user': 'SA', 'qtype': 'dbtable', 'query': 'CustomerAcquisitionTrain', 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'datacache': 1, 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'originalCacheFilePath': '/FileStore/platform/sampleData/620cc1551e8d2b69593f1790/part-00000-25bc40d5-9529-44e5-9075-890202253119-c000.csv'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run cachemanualfeHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "City"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "ARVADA", "max": "LITTLETON", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Product_Category"}], "feature": "Product_Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Category"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Product_Sub_Category"}], "feature": "Product_Sub_Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "Appliances", "max": "Telephones and Communication", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Sub_Category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "327", "mean": "1.07", "stddev": "0.27", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "327", "mean": "1610.9", "stddev": "652.2", "min": "2", "max": "2395", "missing": "0"}, "updatedLabel": "Customer_ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Number", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "327", "mean": "100.0", "stddev": "0.0", "min": "100", "max": "100", "missing": "0"}, "updatedLabel": "Store_Number"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Customer_Segment"}], "feature": "Customer_Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "Consumer", "max": "Small Business", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer_Segment"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "First_Name"}], "feature": "First_Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "ADXXXX", "max": "YVXXXX", "missing": "0", "distinct": "96"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "First_Name"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Last_Name"}], "feature": "Last_Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "ACXXXX", "max": "ZIXXXXXXX", "missing": "0", "distinct": "116"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Last_Name"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Address"}], "feature": "Address", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "1011 S VALENTIA ST", "max": "974 RACINE ST", "missing": "0", "distinct": "125"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Address"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "State"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "CO", "max": "CO", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Zip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "327", "mean": "80166.54", "stddev": "93.3", "min": "80003", "max": "80249", "missing": "0"}, "updatedLabel": "Zip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DriveTime", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "327", "mean": "13.58", "stddev": "8.09", "min": "0.65", "max": "30.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DriveTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length_of_Residense", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "327", "mean": "9.83", "stddev": "9.13", "min": "0", "max": "42", "missing": "0"}, "updatedLabel": "Length_of_Residense"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MOR_BANK_UPSCALE_MERCHANDISE_BUYER", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "327", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "MOR_BANK_UPSCALE_MERCHAND..."}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "MOSAIC_HOUSEHOLD"}], "feature": "MOSAIC_HOUSEHOLD", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "A01", "max": "U00", "missing": "0", "distinct": "42"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC_HOUSEHOLD"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "MOSAIC_DESCRIPTION"}], "feature": "MOSAIC_DESCRIPTION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "Autumn Years", "max": "Young City Solos", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC_DESCRIPTION"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "327", "mean": "-104.91", "stddev": "0.07", "min": "-105.093008", "max": "-104.750245", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "327", "mean": "39.7", "stddev": "0.05", "min": "39.570823", "max": "39.833223", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "327", "mean": "100.0", "stddev": "0.0", "min": "100", "max": "100", "missing": "0"}, "updatedLabel": "Store_ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "327", "mean": "-104.91", "stddev": "0.0", "min": "-104.905387", "max": "-104.905387", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "327", "mean": "39.7", "stddev": "0.0", "min": "39.699065", "max": "39.699065", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lat"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Channel"}], "feature": "Channel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "327", "mean": "", "stddev": "", "min": "Catalog", "max": "eCommerce", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Channel"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run cachemanualfeHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(manualfe, ["City_stringindexer", "Product_Category_stringindexer", "Product_Sub_Category_stringindexer", "Count", "Customer_ID", "Store_Number", "Customer_Segment_stringindexer", "First_Name_stringindexer", "Last_Name_stringindexer", "Address_stringindexer", "State_stringindexer", "Zip", "DriveTime", "Length_of_Residense", "MOR_BANK_UPSCALE_MERCHANDISE_BUYER", "MOSAIC_HOUSEHOLD_stringindexer", "MOSAIC_DESCRIPTION_stringindexer", "Customer_Lon", "Customer_Lat", "Store_ID", "Store_Lon", "Store_Lat"], "Channel_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

